# Data reduction for Amor

In this notebook, we will look at the reduction workflow for reflectometry data collected from the PSI
[Amor](https://www.psi.ch/en/sinq/amor) instrument.
This is a living document and there are plans to update this as necessary with changes in the data reduction methodology and code.

We will begin by importing the modules that are necessary for this notebook and loading the data.
The `sample.nxs` file is the experimental data file of interest,
while `reference.nxs` is the reference measurement of the neutron supermirror.

In [ ]:
import scipp as sc
import scippneutron as scn
from scippneutron.tof import conversions
from ess import amor, reflectometry as refl

## Loading the data

We first load the `sample.nxs` file and perform some early preprocessing using the `amor.io.load` function.

In [ ]:
sample = amor.io.load(amor.data.get_path("sample.nxs"))
sample

By simply plotting the daat, we get a first glimpse into the data contents.

In [ ]:
sc.plot(sample)

It is also possible to show the instrument view using

In [ ]:
scn.instrument_view(sample)

## Coordinate transformation graph

To compute the `wavelength`, scattering angle `two_theta`, and the `Q` vector for our data,
we construct a coordinate transformation graph.
It is based on classical conversions from `tof` and pixel `position` to `wavelength`, `two_theta` and `Q`,
but comprises a number of modifications.

The computation of the scattering angle `two_theta` ignores the $x$ component of the scattered beam ($L_{2}$) vector,
and includes a correction for the Earth's gravitational field which bends the flight path of the neutrons.
The angle can be found using the following
[expression](https://docs.mantidproject.org/v3.9.0/algorithms/Q1D-v2.html#algm-q1d)

$$\theta = \frac{1}{2}\arcsin\left(\frac{\sqrt{x^{2} + \left(y + \frac{g m_{\rm n}}{2 h^{2}} \lambda^{2} L_{2}^{2}\right)^{2}}}{L_{2}}\right)$$

where $m_{\rm n}$ is the neutron mass,
$g$ is the acceleration due to gravity,
and $h$ is Planck's constant
(this assumes the neutrons are all travelling horizontally at the position of the sample).

The conversion graph is defined in the reflectometry module,
and can be obtained via

In [ ]:
graph = refl.conversions.reflectometry_graph()
graph

In [ ]:
sc.show_graph(graph, simplified=True)

## Computing the wavelength

To compute the wavelength of the neutrons,
we request the `wavelength` coordinate from the `transform_coords` method by supplying our graph defined above
(see [here](https://scipp.github.io/scippneutron/user-guide/coordinate-transformations.html)
for more information about using `transform_coords`)

In [ ]:
wavelength = sample.transform_coords(["wavelength"], graph=graph)
wavelength

In [ ]:
wavelength.bins.concatenate('detector_id').plot()

## Compute the Q vector

Using the same method, we can compute the $Q$ vector,
which now depends on both detector position (id) and wavelength

In [ ]:
q_with_gravity = sample.transform_coords(["Q"], graph=graph)
q_with_gravity

In [ ]:
q_edges = sc.linspace(dim='Q', start=0.008, stop=0.08,
                      num=201, unit='1/angstrom')
q_binned_with_g = sc.bin(q_with_gravity, edges=[q_edges])
q_summed_with_g = q_binned_with_g.sum('detector_id')
sc.plot(q_summed_with_g["wavelength", 0], norm="log")